In [1]:
from langchain_community.graphs import OntotextGraphDBGraph
from langchain_classic.chains import OntotextGraphDBQAChain
from huggingface_hub import login
from dotenv import load_dotenv
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import os
import pandas as pd

/home/sajalpaudyal/Documents/Programming_Works/rdf_to_graphRAG_pipeline/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
load_dotenv()
HF_TOKEN  = os.getenv("SAJAL_HF_TOKEN")


In [3]:
os.environ["GRAPHDB_USERNAME"] = "admin"
os.environ["GRAPHDB_PASSWORD"] = "root"

In [4]:
graph = OntotextGraphDBGraph(
    query_endpoint="http://localhost:7200/repositories/lemonde_top_100_books",   
    local_file="../rules/lemonde_books.ttl",
    local_file_format="turtle"
)



In [5]:
login = login(token=HF_TOKEN)

In [6]:


llm = HuggingFaceEndpoint(
    model="zai-org/GLM-4.7",
    task="conversational",
    do_sample=False,
    repetition_penalty=1.03,
)



In [7]:
chat_model = ChatHuggingFace(llm=llm)

In [8]:
chain = OntotextGraphDBQAChain.from_llm(
    chat_model, 
    graph = graph,
    verbose = True,
    allow_dangerous_requests = True
)

In [12]:
chain.invoke({chain.input_key: "Who wrote For Whom the Bell Tolls?"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX : <http://www.semanticweb.org/sajalpaudyal/ontologies/2026/lemonde100/>

SELECT ?author
WHERE {
  ?book a :Book ;
        :title "For Whom the Bell Tolls" ;
        :hasAuthor ?author .
}

> Finished chain.


'Based on the information provided, Ernest Hemingway wrote For Whom the Bell Tolls.'

In [10]:
chain.invoke({chain.input_key: "What is the rating of The Stranger?"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX : <http://www.semanticweb.org/sajalpaudyal/ontologies/2026/lemonde100/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?rating
WHERE {
  ?book a :Book ;
        :title "The Stranger" ;
        :goodReadRating ?rating .
}

> Finished chain.


'The rating of The Stranger is 4.03.'